<a href="https://colab.research.google.com/github/vlasvlasvlas/Colabs/blob/main/Dynamic_Barchart_Index_Audit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# ------------------
# all in one code:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io
from ipywidgets import widgets, VBox, HBox, Button, HTML, Layout
from google.colab import files

# Crear widgets para los valores de entrada y mensaje de error
inputvalue1 = widgets.FloatText(value=30, description='InputValue1:', style={'description_width': 'initial'})
inputvalue2 = widgets.FloatText(value=30, description='InputValue2:', style={'description_width': 'initial'})
inputvalue3 = widgets.FloatText(value=10, description='InputValue3:', style={'description_width': 'initial'})
inputvalue4 = widgets.FloatText(value=30, description='InputValue4:', style={'description_width': 'initial'})
error_message = HTML(value="")

# Botón para cargar archivo y actualizar el gráfico
upload_button = Button(description="Load CSV File", layout=Layout(width='auto', margin='10px 0 10px 0'))
update_button = Button(description="Update Chart", layout=Layout(width='auto', margin='10px 0 0 0'))

# Contenedor para el nombre del archivo
filename_label = HTML(value="No file uploaded.")

# Cargar archivo y actualizar gráfico
def upload_file(_):
    global data, filename  # Declarar como global si necesitas usar estas variables fuera de esta función
    uploaded = files.upload()
    if uploaded:
        filename = next(iter(uploaded))
        data = pd.read_csv(io.BytesIO(uploaded[filename]))
        filename_label.value = f"Archivo cargado: {filename}"
        update_graph(None)  # Llamar a la actualización del gráfico directamente después de cargar

# Función para actualizar el gráfico y validar la suma
def update_graph(_):
    if 'data' not in globals():
        error_message.value = "<font color='red'>Please upload a file first.</font>"
        return
    total_input_value = inputvalue1.value + inputvalue2.value + inputvalue3.value + inputvalue4.value
    if total_input_value > 100:
        error_message.value = "<font color='red'>Error: The sum of the values must not exceed 100. Currently sum: {:.2f}</font>".format(total_input_value)
        return
    else:
        error_message.value = ""

    data['indice'] = (data.iloc[:,1] * inputvalue1.value/100 +
                      data.iloc[:,2] * inputvalue2.value/100 +
                      data.iloc[:,3] * inputvalue3.value/100 +
                      data.iloc[:,4] * inputvalue4.value/100 )

    top_50 = data.sort_values('indice', ascending=False).head(50)
    top_50['country'] = top_50['country'].str.slice(0, 40) + ' (' + (top_50.reset_index().index + 1).astype(str) + ')'

    plt.figure(figsize=(12, 15))
    color_palette = 'coolwarm' # https://seaborn.pydata.org/tutorial/color_palettes.ht
    bar_plot = sns.barplot(x='indice', y='country', data=top_50, palette=color_palette)

    for index, value in enumerate(top_50['indice']):
        text = bar_plot.text(value, index, f'{value:.2f}', color='black', va='center', fontweight='bold', ha='right')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

    plt.title(f'Index Calculation for Data Analysis ({filename})')
    plt.xlabel('Index')
    plt.ylabel('Country')
    plt.xlim(0, top_50['indice'].max() * 1.1)
    plt.show()

# Vincular el botón con las funciones de carga de archivo y actualización
upload_button.on_click(upload_file)
update_button.on_click(update_graph)

# Widgets para título y descripción
title = HTML(value='<h2>Index Calculation for Data Analysis</h2>', layout=Layout(margin='0 0 5px 0'))

description = HTML(value='''
<h3>Analysis Process</h3>
<p>Follow these steps to calculate the index and view the results:</p>
<ol>
     <li>Upload the CSV file using the 'Upload File' button.</li>
     <li>Set the input values for the four indicators: InputValue1, InputValue2, InputValue3, and InputValue4.</li>
     <li>Press the 'Refresh Chart' button to view the bar graph of the uploaded data.</li>
</ol>
<p>Make sure that each file contains at least the five necessary columns: 'country', and the 4 columns of data for which the index needs to be generated.</p>
''', layout=Layout(margin='0 0 15px 0'))


# Mostrar los widgets y los mensajes
input_widgets = VBox([title, description, filename_label, upload_button, inputvalue1, inputvalue2, inputvalue3, inputvalue4, update_button, error_message])
display(input_widgets)
